In [1]:
import csv
import numpy as np
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import pickle
import pandas as pd
#import tensorflow as tf
import json
from sklearn.model_selection import train_test_split
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
# data_dir='../data/Current Data/'
data_dir = r"C:\work\dev\dECMT_src\data_all\COVID19_Data\Current\\"

In [3]:
CategoricalFeatures=[]
NumericFeatures=[]

demographics

In [4]:
df_demographics=pd.read_csv(data_dir+'REACT_Demographics'+'.csv')
#df_demographics.info()

In [5]:
#df_demographics.head()

In [6]:
#df_demographics.columns

In [7]:
CategoricalFeatures+=['GENDER','ETHNIC_GROUP', 'SMOKING_HISTORY']
NumericFeatures+=['PATIENT_AGE']

In [8]:
studyid_data_dic=dict()
for studyid,age,gender,ethnic,smoking in df_demographics[['STUDY_ID','PATIENT_AGE','GENDER','ETHNIC_GROUP', 'SMOKING_HISTORY']].values:
    if age is not np.nan and gender is not np.nan and ethnic is not np.nan and smoking is not np.nan:
        if studyid not in studyid_data_dic:
            studyid_data_dic[studyid]=dict()
        studyid_data_dic[studyid]['PATIENT_AGE']=age
        studyid_data_dic[studyid]['GENDER']=str(gender)
        studyid_data_dic[studyid]['ETHNIC_GROUP']=ethnic
        studyid_data_dic[studyid]['SMOKING_HISTORY']=smoking
#studyid_data_dic

Comorbidities

In [9]:
df_comorbidities=pd.read_csv(data_dir+'REACT_Comorbidities_unpivoted'+'.csv')
#df_demographics.info()

In [10]:
comorbidities=list(df_comorbidities.COMORBIDITY.unique())
NumericFeatures+=comorbidities
for studyid in studyid_data_dic:
    for co in comorbidities:
        studyid_data_dic[studyid][co]=0

In [11]:
for studyid,co,status in df_comorbidities.values:
    if studyid in studyid_data_dic:
        studyid_data_dic[studyid][co]=status
#studyid_data_dic

label: RespiratorySupportRequired or not

In [12]:
df_Vitalsigns_Categorical=pd.read_csv(data_dir+'REACT_Vitalsigns_Categorical'+'.csv')
#df_Vitalsigns_Categorical.info()

In [13]:
#df_Vitalsigns_Categorical.head()

In [14]:
CategorySet=dict()
CategorySet[0]=set(['Air - Not Supported'])
CategorySet[1]=set([
    'Nasal Specs', 'Face Mask', 'Venturi Mask',
    'Non-Rebreath Mask', 'Invasive Ventilation', 'Optiflow / Hi Flow',
    'NIV - CPAP face mask', 'NIV - CPAP full face mask',
    'NIV - BIPAP face mask', 'NIV - CPAP nasal mask',
    'NIV - BIPAP nasal mask', 'NIV - BIPAP full face mask',
    'Trachy Mask'
])
CategoryDictionary=dict([(item,label) for label in CategorySet for item in CategorySet[label]])
CategoryDictionary

{'Air - Not Supported': 0,
 'Trachy Mask': 1,
 'NIV - BIPAP full face mask': 1,
 'Venturi Mask': 1,
 'Face Mask': 1,
 'Invasive Ventilation': 1,
 'NIV - CPAP face mask': 1,
 'NIV - BIPAP nasal mask': 1,
 'NIV - CPAP nasal mask': 1,
 'Optiflow / Hi Flow': 1,
 'Nasal Specs': 1,
 'Non-Rebreath Mask': 1,
 'NIV - BIPAP face mask': 1,
 'NIV - CPAP full face mask': 1}

In [15]:
df_label=df_Vitalsigns_Categorical[df_Vitalsigns_Categorical.PARAMETER=='Respiratory Support'][['STUDY_ID','UNITFROM_DATETIME','VALUE']]

In [16]:
from datetime import datetime

label_studyid_time_dic=dict()
for study_id,start_datetime,label in df_label.values:
    start_datetime=start_datetime.replace('.000','')#need to be fixed some datetime end with .000
    start_datetime=datetime.strptime(start_datetime,'%Y-%m-%d %H:%M:%S')
    label_category=CategoryDictionary[label]
    if study_id not in label_studyid_time_dic:
        label_studyid_time_dic[study_id]=[start_datetime,label_category]
    else:
        curr_start_datetime,curr_label_category=label_studyid_time_dic[study_id]
        if label_category>=curr_label_category and curr_start_datetime>start_datetime:
            label_studyid_time_dic[study_id]=[start_datetime,label_category]

#label_studyid_time_dic

In [17]:
for studyid in studyid_data_dic:
    if studyid not in label_studyid_time_dic:
        studyid_data_dic[studyid]['TIME_POINT']=None
        studyid_data_dic[studyid]['PREDICTION_LABEL']=0
    else:
        start_datetime,label_category=label_studyid_time_dic[studyid]
        studyid_data_dic[studyid]['TIME_POINT']=start_datetime
        studyid_data_dic[studyid]['PREDICTION_LABEL']=label_category

labresults

In [18]:
df_labresults=pd.read_csv(data_dir+'REACT_LabResults'+'.csv')
#df_labresults.info()

In [19]:
#df_labresults.head()

In [20]:
testcode_lst=list(df_labresults.REACT_TESTCODE.unique())
testcode2id=dict([(testcode_lst[i],i) for i in range(len(testcode_lst))])

In [21]:
from datetime import datetime

studyid_testcode_time_numeric_dic=dict()
for studyid,test_time,testcode,numeric,lower,upper in df_labresults[[
    'STUDY_ID','PATHOLOGY_SPECIMEN_DATE','REACT_TESTCODE',
    'PATHOLOGY_RESULT_NUMERIC','LOWER_RANGE','UPPER_RANGE']].values:
    test_time=datetime.strptime(test_time,'%Y-%m-%d %H:%M:%S')
    if studyid not in studyid_testcode_time_numeric_dic:
        studyid_testcode_time_numeric_dic[studyid]=[]
    studyid_testcode_time_numeric_dic[studyid].append([test_time,testcode,numeric])
#studyid_testcode_time_numeric_dic

In [22]:
NumericFeatures+=[testcode+suffix for testcode in testcode_lst for suffix in ['_MIN','_MAX','_MEAN']]

In [23]:
for studyid in studyid_data_dic:
    for testcode in testcode_lst:
        for suffix in ['_MIN','_MAX','_MEAN']:
            studyid_data_dic[studyid][testcode+suffix]=None
    time_point=studyid_data_dic[studyid]['TIME_POINT']
    if studyid in studyid_testcode_time_numeric_dic:
        individual_labresult_dic=dict()
        for test_time,testcode,numeric in studyid_testcode_time_numeric_dic[studyid]:
            if time_point is None or test_time<time_point:
                if testcode not in individual_labresult_dic:
                    individual_labresult_dic[testcode]=[]
                individual_labresult_dic[testcode].append(numeric)
        for testcode in individual_labresult_dic:
            studyid_data_dic[studyid][testcode+'_MIN']=np.min(individual_labresult_dic[testcode])
            studyid_data_dic[studyid][testcode+'_MAX']=np.min(individual_labresult_dic[testcode])
            studyid_data_dic[studyid][testcode+'_MEAN']=np.mean(individual_labresult_dic[testcode])

data preprocessing

In [24]:
studyid_lst=list(studyid_data_dic.keys())
#studyid_lst

In [25]:
X_cols=[]

In [26]:
from sklearn.preprocessing import OneHotEncoder

X_categorical=[[studyid_data_dic[studyid][k] for k in CategoricalFeatures] for studyid in studyid_lst]

enc = OneHotEncoder(handle_unknown='ignore')
X_onehot=enc.fit_transform(X_categorical).toarray()

X_cols+=list(enc.get_feature_names(CategoricalFeatures))

In [27]:
X_numeric=[[studyid_data_dic[studyid][k] for k in NumericFeatures] for studyid in studyid_lst]
X_cols+=NumericFeatures

In [28]:
X_missing_value=np.concatenate((X_onehot,X_numeric), axis=1)

In [29]:
Y=np.array([studyid_data_dic[studyid]['PREDICTION_LABEL'] for studyid in studyid_lst])

In [30]:
balance_ratio=len([_ for v in Y if v==0])/len([_ for v in Y if v==1])
balance_ratio

0.826865671641791

from functools import reduce

XY_missing_value_balanced=np.array(
    reduce(lambda a,b:a+b,[[list(x)+[y]]*(balance_ratio if y==1 else 1) for x,y in zip(X_missing_value,Y)])
)
#XY_missing_value_balanced

fill in nan

In [31]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

X=imputer.fit_transform(X_missing_value[:,:-1])

Prediction Classification task (RespiratorySupportRequired:1 or not:0)

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

models={
    "Logit":LogisticRegression(solver="lbfgs"),
    "MLP":MLPClassifier(max_iter=3000,solver='adam', alpha=1e-3,hidden_layer_sizes=(64)),
    "DecisionTree":DecisionTreeClassifier(),
    "RandomForest":RandomForestClassifier(n_estimators=100),
    "AdaBoost":AdaBoostClassifier(n_estimators=100),
    "GradientBoost":GradientBoostingClassifier(n_estimators=100),
    "XGBoost":XGBClassifier(n_estimators=100)
}

In [33]:
from sklearn.model_selection import KFold

from sklearn.metrics import average_precision_score

ap_result_lst=[]

kf = KFold(n_splits=10)

n_class=2

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    Y_test = np.zeros([len(y_test),n_class])
    for i in range(len(y_test)):
        Y_test[i,y_test[i]]=1
    ap_result_lst.append(
        [average_precision_score(Y_test,models[model_name].fit(X_train,y_train).predict_proba(X_test))\
                for model_name in models])

print(pd.DataFrame(ap_result_lst,columns=[model_name for model_name in models]))

      Logit       MLP  DecisionTree  RandomForest  AdaBoost  GradientBoost  \
0  0.555315  0.550588      0.472015      0.552513  0.505853       0.551033   
1  0.524854  0.590504      0.547829      0.521746  0.471443       0.574415   
2  0.650982  0.617922      0.525199      0.623482  0.570411       0.584342   
3  0.519934  0.480721      0.480639      0.629975  0.526477       0.604769   
4  0.602486  0.537256      0.513567      0.508478  0.504359       0.585612   
5  0.608466  0.619951      0.525754      0.502282  0.538252       0.528892   
6  0.543017  0.540695      0.621949      0.654360  0.611511       0.650599   
7  0.708817  0.627542      0.512391      0.575694  0.585472       0.607442   
8  0.499619  0.548208      0.548005      0.622950  0.559462       0.596080   
9  0.588180  0.537157      0.506618      0.550700  0.600559       0.616327   

    XGBoost  
0  0.533504  
1  0.577811  
2  0.603359  
3  0.582111  
4  0.560394  
5  0.573822  
6  0.669118  
7  0.643567  
8  0.612696  
9

In [34]:
print(pd.DataFrame(np.mean(ap_result_lst,axis=0,keepdims=True),columns=[model_name for model_name in models]))

      Logit       MLP  DecisionTree  RandomForest  AdaBoost  GradientBoost  \
0  0.580167  0.565054      0.525397      0.574218   0.54738       0.589951   

    XGBoost  
0  0.595625  


In [35]:
class_names=['Not required','Required']

In [36]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver="lbfgs").fit(X, Y)

In [37]:
import shap

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.LinearExplainer(clf,X)
shap_values = explainer.shap_values(X)

# summarize the effects of all the features
shap.summary_plot(shap_values, X, feature_names=X_cols, plot_type="bar",class_names=class_names)

ModuleNotFoundError: No module named 'shap'

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier().fit(X, Y)

In [ ]:
feature_importance_dic=dict(zip(X_cols,clf.feature_importances_))
from sklearn.feature_selection import SelectFromModel
for k,v in sorted(
    [(X_cols[i],feature_importance_dic[X_cols[i]]) for i in SelectFromModel(clf, prefit=True, max_features=20).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
):
    print(k,':',v)

In [ ]:
import shap

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X)

# summarize the effects of all the features
shap.summary_plot(shap_values, X, feature_names=X_cols, plot_type="bar",class_names=class_names)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier().fit(X, Y)

In [ ]:
feature_importance_dic=dict(zip(X_cols,clf.feature_importances_))
from sklearn.feature_selection import SelectFromModel
for k,v in sorted(
    [(X_cols[i],feature_importance_dic[X_cols[i]]) for i in SelectFromModel(clf, prefit=True, max_features=20).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
):
    print(k,':',v)

In [ ]:
import shap

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X)

# summarize the effects of all the features
shap.summary_plot(shap_values, X, feature_names=X_cols, plot_type="bar",class_names=class_names)

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier(n_estimators=100).fit(X, Y)

In [ ]:
feature_importance_dic=dict(zip(X_cols,clf.feature_importances_))
from sklearn.feature_selection import SelectFromModel
for k,v in sorted(
    [(X_cols[i],feature_importance_dic[X_cols[i]]) for i in SelectFromModel(clf, prefit=True, max_features=20).get_support(indices=True)],
    key=lambda x:x[1],
    reverse=True
):
    print(k,':',v)